In [13]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import os
from src.progress_bar import printProgressBar

for dirname, _, filenames in os.walk('../../data/lyrics'):
    for filename in filenames:
        print(os.path.join(dirname, filename).replace("\\", "/"))

print(50 * '-')

for dirname, _, filenames in os.walk('../../data/billboard'):
    for filename in filenames:
        print(os.path.join(dirname, filename).replace("\\", "/"))




../../data/lyrics/artist_song_lyrics.csv
../../data/lyrics/artist_song_lyrics_new.csv
../../data/lyrics/bb-t100-lyrics.csv
../../data/lyrics/bb-t100-lyrics_new.csv
../../data/lyrics/bb_t100_lyrics_en.csv
../../data/lyrics/bb_t100_lyrics_en_new.csv
../../data/lyrics/bb_t100_lyrics_en_v2.csv
../../data/lyrics/corona-lyrics.csv
../../data/lyrics/covid_lyrics_bb.csv
../../data/lyrics/lyrics_invalid.json
../../data/lyrics/lyrics_invalid_new.json
../../data/lyrics/lyrics_invalid_updated.csv
../../data/lyrics/lyrics_invalid_updated_new.csv
../../data/lyrics/backups/artist_song_lyrics_bak.csv
../../data/lyrics/backups/bb-t100-lyrics.csv
../../data/lyrics/backups/bb-t100-lyrics_new.csv
../../data/lyrics/backups/bb-t100-lyrics_old.csv
../../data/lyrics/backups/bb_t100_lyrics_en.csv
../../data/lyrics/backups/bb_t100_lyrics_en_v1.csv
../../data/lyrics/backups/bb_t100_lyrics_en_v2.csv
../../data/lyrics/backups/bb_t100_lyrics_en_v3.csv
../../data/lyrics/backups/corona-lyrics.csv
../../data/lyrics/ba

In [14]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 30)
lyrics = pd.read_csv('../../data/lyrics/bb-t100-lyrics_new.csv', index_col=0)
lyrics.head()

,billboard_id,lyrics_id,artist,first_artist,song,weeks_on_chart,peak_rank,first_appearance,lyrics,url,genius_id,genius_primary_artist,genius_description,genius_annotations,genius_comments,length,word_count,language,language_score
0,0,0,Ariana Grande,Ariana Grande,"Thank U, Next",28.0,1.0,2019-01-05,Thought I'd end up with Se...,https://genius.com/Ariana-...,4063065.0,"Ariana Grande, 0 songs",On the lead single and tit...,[('One taught me love\n On...,['The Mac shoutout has me ...,2411,460,en,0.999997
1,1,1,Halsey,Halsey,Without Me,52.0,1.0,2019-01-05,Found you when your heart ...,https://genius.com/Halsey-...,3977187.0,"Halsey, 0 songs",“Without Me” is the first ...,[('Gave love ’bout a hundr...,['The queen is ready to sn...,2096,435,en,0.999995
2,2,2,Mariah Carey,Mariah Carey,All I Want For Christmas I...,43.0,1.0,2019-01-05,I don't want a lot for Chr...,https://genius.com/Mariah-...,204233.0,"Mariah Carey, 0 songs",“All I Want For Christmas ...,[('I don’t need to hang my...,['i really like this song'...,1919,388,en,0.999997
3,3,3,Travis Scott,Travis Scott,Sicko Mode,52.0,1.0,2019-01-05,"Astro, yeah\nSun is down, ...",https://genius.com/Travis-...,3876994.0,"Travis Scott, 0 songs",“SICKO MODE” refers to Tra...,[('She’s in love with who ...,['HAD ME OUT LIKE A LIGHT ...,3946,771,en,0.999998
4,4,4,Post Malone & Swae Lee,Post Malone,Sunflower (Spider-Man: Int...,53.0,1.0,2019-01-05,1. Lil Nas X - INDUSTRY BA...,https://genius.com/Spotify...,3993850.0,Post Malone & Swae Lee,“Sunflower” marks the seco...,[('Then you’re left in the...,['Me enjoying “Sunflower” ...,2623,545,en,0.999998


In [15]:
bb_t100 = pd.read_csv('../../data/billboard/billboard.csv', index_col=0)
bb_t100.rename(columns={'id': 'billboard_id'}, inplace=True)
bb_t100.head()

,billboard_id,song,artist,peak_rank,rank_last_week,weeks_on_chart,rank,date
0,0,"Thank U, Next",Ariana Grande,1,1,8,1,2019-01-05
1,1,Without Me,Halsey,2,2,12,2,2019-01-05
2,2,All I Want For Christmas I...,Mariah Carey,3,7,30,3,2019-01-05
3,3,Sicko Mode,Travis Scott,1,3,21,4,2019-01-05
4,4,Sunflower (Spider-Man: Int...,Post Malone & Swae Lee,4,4,10,5,2019-01-05


In [16]:
merge_cols = lyrics.columns.difference(bb_t100.columns)
merge_cols = merge_cols.tolist()
merge_cols.append('billboard_id')
merge_cols

['first_appearance',
 'first_artist',
 'genius_annotations',
 'genius_comments',
 'genius_description',
 'genius_id',
 'genius_primary_artist',
 'language',
 'language_score',
 'length',
 'lyrics',
 'lyrics_id',
 'url',
 'word_count',
 'billboard_id']

In [17]:
merge_cols = ['language', 'language_score', 'lyrics_id', 'billboard_id', 'first_appearance', ]
bb_t100 = bb_t100.merge(lyrics[merge_cols], how='left', left_on='billboard_id', right_on='billboard_id', copy=False)
bb_t100 = bb_t100.sort_values(by=['artist', 'song', 'lyrics_id']).interpolate(method='pad')
bb_t100['lyrics_id'] = pd.to_numeric(bb_t100['lyrics_id'], downcast='integer')
bb_t100['date'] = pd.to_datetime(bb_t100['date'], format='%Y-%m-%d')
bb_t100.sort_values(by='billboard_id', inplace=True)
bb_t100 = bb_t100[['billboard_id',
                   'lyrics_id',
                   'song',
                   'artist',
                   'peak_rank',
                   'rank_last_week',
                   'weeks_on_chart',
                   'rank',
                   'date',
                   'first_appearance',
                   'language',
                   'language_score']]
bb_t100.head()

,billboard_id,lyrics_id,song,artist,peak_rank,rank_last_week,weeks_on_chart,rank,date,first_appearance,language,language_score
0,0,0,"Thank U, Next",Ariana Grande,1,1,8,1,2019-01-05,2019-01-05,en,0.999997
1,1,1,Without Me,Halsey,2,2,12,2,2019-01-05,2019-01-05,en,0.999995
2,2,2,All I Want For Christmas I...,Mariah Carey,3,7,30,3,2019-01-05,2019-01-05,en,0.999997
3,3,3,Sicko Mode,Travis Scott,1,3,21,4,2019-01-05,2019-01-05,en,0.999998
4,4,4,Sunflower (Spider-Man: Int...,Post Malone & Swae Lee,4,4,10,5,2019-01-05,2019-01-05,en,0.999998


In [18]:
bb_t100_en = bb_t100.loc[bb_t100['language'] == 'en']
# bb_t100_en.reset_index(drop=True, inplace=True)
bb_t100_en.to_csv('../../data/billboard/bb_t100_en_new.csv', encoding='utf-8')

lyrics_en = lyrics.loc[lyrics['language'] == 'en']
# lyrics_en.reset_index(drop=True, inplace=True)
lyrics_en.to_csv('../../data/lyrics/bb_t100_lyrics_en_new.csv', encoding='utf-8')